In [1]:
import numpy as np
import rqcopt as oc
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
import jax
from jax import config
config.update("jax_enable_x64", True)
import jax.numpy as jnp
import scipy

In [105]:
# setup for problem
# get the setup for the problem
from opentn.transformations import create_kitaev_liouvillians, exp_operator_dt, factorize_psd, super2choi
d, N, gamma = 2, 4, 1
tau = 1
dim = d**N
Lvec, Lvec_odd, Lvec_even, Lnn = create_kitaev_liouvillians(N=N, d=d, gamma=gamma)
superops_exp = []
for i, op in enumerate([Lvec, Lvec_odd, Lvec_even]):
    if i == 1:
        superops_exp.append(exp_operator_dt(op, tau/2, 'jax'))
    else:
        superops_exp.append(exp_operator_dt(op, tau, 'jax'))
exp_Lvec, exp_Lvec_odd, exp_Lvec_even = superops_exp

# 2 site superoperator ansatz
from opentn.transformations import lindbladian2super, create_supertensored_from_local
superop_nn = lindbladian2super(Li=[Lnn])
exp_nn_odd = exp_operator_dt(superop_nn, tau=tau/2, library='jax')
exp_nn_even = exp_operator_dt(superop_nn, tau=tau, library='jax')

In [71]:
# TODO: missing to create the model that takes 3 ws and turns them into 3 full Ys.
# TODO: model Zs does something similar but only for 1 layer.
# the one for the middle layer should somehow look different in any case, because it needs to be shifted with the periodic boundary condition.
# since the exp is anyways left without the BC, for now I think I can just ommit it (as usual
# TODO: the cost function should have the input as isometries.

In [72]:
from opentn.states.qubits import X

Inn = np.eye(d**2)
I_vec = lindbladian2super(Li=[Inn]) # this is all zeros
even_test = np.kron(I_vec, exp_nn_even)
print(np.allclose(even_test, exp_Lvec_even)) # as expected they are not the same, but how about after permutation? NO! since the first thing is all zeros
def permute_operation(U: np.ndarray, perm):
    """
    Find the representation of a matrix after permuting lattice sites.
    """
    nsites = len(perm)
    assert U.shape == (2**nsites, 2**nsites)
    perm = list(perm)
    U = np.reshape(U, (2*nsites) * (2,))
    U = np.transpose(U, perm + [nsites + p for p in perm])
    U = np.reshape(U, (2**nsites, 2**nsites))
    return U


permute_operation(np.kron(np.eye(2),X), [1,0])


False


array([[0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
       [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j]])

In [77]:
# how would we vectorize the identity?
from opentn.transformations import vectorize_dissipative, vectorize_hamiltonian, dissipative2liouvillian_full
print(np.allclose(vectorize_dissipative(Inn), np.zeros((d**4,d**4))), np.allclose(vectorize_hamiltonian(Inn), np.zeros((d**4,d**4))))
# as wrote out on paper, the identity "vectorized" is just zero
# instead maybe I should write it like just a big identity, reshufle and see if this is e
print(dissipative2liouvillian_full(L=Lnn, i=1, N=N, num_sites=2).shape)

True True
(256, 256)


In [106]:
from opentn.transformations import get_indices_supertensored2liouvillianfull
I_rest = np.eye(d**4,d**4)
# corresponds to site order: (N-1, 0), (1, 2)
even_full_test = np.kron(I_rest, exp_nn_even) 
print(even_full_test.shape)
source_indices, destination_indices = get_indices_supertensored2liouvillianfull(N)
swaped_test = np.reshape(even_full_test, newshape=[d]*4*N) # 2 for each side (normal and conjugate -> from vectorization)
swaped_test = np.moveaxis(swaped_test, source=source_indices, destination=destination_indices)
# extra step to enforce PBC
idx_permuted = permute_cyclic(list(range(N)), 1) + permute_cyclic(list(range(N,2*N)), 1)
swaped_test = np.transpose(swaped_test, idx_permuted + list(np.array(idx_permuted) + 2*N))
swaped_test = np.reshape(swaped_test, newshape=even_full_test.shape)
np.allclose(swaped_test, exp_Lvec_even)

(256, 256)


True